In [1]:
 !pip install transformers --upgrade
 !pip install wordninja

     |████████████████████████████████| 890kB 4.5MB/s 
     |████████████████████████████████| 3.0MB 14.9MB/s 
     |████████████████████████████████| 1.1MB 48.8MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=10c6db3ab73609ec3020e320b549199f8de51810f82b971021da6d1ef3e398d6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 542kB 4.3MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp36-none-any.whl size=541553 sha256=6908d4bbd77eac59d9a030590366505c8dd0b6afc86744348c51fd84d961f4de
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built wordninja


In [2]:
from textblob import TextBlob
import wordninja
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFBertLMHeadModel
from string import punctuation
from tensorflow.data import Dataset
import tensorflow as tf
import spacy

spacy_nlp = spacy.load('en_core_web_sm')

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [3]:
#sample = pd.read_csv('./drive/My Drive/mentalhealth/SampleSubmission.csv')

In [4]:
train = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/nlp/master/mental_health.csv')
train.head()

,ID,text,label
0,0,I feel that it was better I dieAm happy,Depression
1,1,Why do I get hallucinations?,Drugs
2,2,I am stresseed due to lack of financial suppor...,Depression
3,3,Why is life important?,Suicide
4,4,How could I be helped to go through the depres...,Depression


In [5]:
train['text'] = train['text'].apply(lambda row: " ".join([word.translate(str.maketrans('', '', punctuation)) for word in row.split()]))


In [6]:
train['text'] = train['text'].apply(lambda row: " ".join([" ".join(wordninja.split(word)) for word in row.split()]))


In [7]:
train['text'] = train['text'].apply(lambda x: " ".join([str(TextBlob(i).correct()) for i in x.split()]))


In [8]:
train['text'] = train['text'].str.replace('\d+', '')

In [9]:
train['text'] = train['text'].apply(lambda row: " ".join([word for word in row.split() if word not in spacy_stopwords]))

In [10]:
MAX_LEN = train.text.str.len().max()
MAX_LEN

121

In [11]:
labels = pd.get_dummies(train['label'])
labels = tf.convert_to_tensor(labels)
labels

<tf.Tensor: shape=(616, 4), dtype=uint8, numpy=
array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       ...,
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0]], dtype=uint8)>

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

bert_model = TFBertLMHeadModel.from_pretrained("bert-base-uncased")

If you want to use `TFBertLMHeadModel` as a standalone, add `is_decoder=True.`


Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertLMHeadModel: ['nsp___cls']
- This IS expected if you are initializing TFBertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertLMHeadModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertLMHeadModel for predictions without further training.


In [13]:
text_encode = tokenizer.batch_encode_plus(train['text'], padding='max_length', return_tensors='tf', max_length=MAX_LEN)

In [14]:
ids = text_encode['input_ids']
tks = text_encode['token_type_ids']
mask = text_encode['attention_mask']

In [18]:
def get_model():
  ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
  mask = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')
  tks = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name='token_type_ids')

  x = bert_model(inputs=ids, attention_mask=mask, token_type_ids=tks)[0]
  x = x[:, 0, :]
  x = tf.keras.layers.Dense(4, activation='sigmoid')(x)
  model = tf.keras.Model(inputs=(ids, mask, tks), outputs=x)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
  return model

In [19]:
model = get_model()
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 121)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 121)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 121)]        0                                            
__________________________________________________________________________________________________
tf_bert_lm_head_model (TFBertLM ((None, 121, 30522), 110104890   input_ids[0][0]                  
                                                                 attention_mask[0][0]  

In [20]:
model.fit((ids, mask, tks), labels, epochs=10)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


InvalidArgumentError: ignored

In [ ]:
predictions = model.predict((ids, mask))
predictions

In [ ]:
# df = pd.DataFrame(predictions, columns=['Depression',	'Alcohol',	'Suicide',	'Drugs'])
# df = pd.concat([sample['ID'], df], axis=1)

# x = df['Alcohol'].copy() #Invert
# df['Alcohol'] = df['Depression'].copy()
# df['Depression'] = x

# x = df['Suicide'].copy() #Invert
# df['Suicide'] = df['Drugs'].copy()
# df['Drugs'] = x

# df.to_csv('health.csv', index=False)
# df.head()